# MIPSGAL のデータを一括ダウンロード
- 引数1: int 型の 0-359 数字 (間に観測対象外のものも含まれるため注意)
- 引数2: str 型の 'p' か 'n'
- 引数3: 保存場所の Path

In [1]:
import sys
import urllib.request

sys.path.append('..')

### 指定したファイルをダウンロード可能か試験

In [2]:
save_path = '/root/data/science/spitzer/mipsgal/'
url = 'https://irsa.ipac.caltech.edu/data/SPITZER/MIPSGAL/images/mosaics24/'
file_name = 'MG0000n005_024.fits'
with urllib.request.urlopen(url+file_name) as u:
    with open(save_path+file_name, 'bw') as o:
        o.write(u.read())

### 以下、作成した関数および使用例

In [3]:
def download_mipsgal_data(l, b, save_path='/root/data/science/spitzer/mipsgal/'):
    '''
    l must be int.
    b must be str, 'p' or 'n'.
    '''
    url = 'https://irsa.ipac.caltech.edu/data/SPITZER/MIPSGAL/images/mosaics24/'
    l = str(l)+'0'
    while len(l)!=4:
        l = '0'+l
        
    file_name = 'MG'+l+b+'005_024.fits'
    with urllib.request.urlopen(url+file_name) as u:
        with open(save_path+file_name, 'bw') as o:
            o.write(u.read())
            
    return

In [4]:
l = [i for i in range(0, 66)]
for i in l:
    download_mipsgal_data(i, 'p')
    download_mipsgal_data(i, 'n')
    
l = [i for i in range(296, 359)]
for i in l:
    download_mipsgal_data(i, 'p')
    download_mipsgal_data(i, 'n')

### /src/utils/data_utils.py に関数を追加した。

In [5]:
from src.utils import  data_utils
download_mipsgal_data(0, 'p')